<a href="https://colab.research.google.com/github/Nnekaa/course-git-blog-project/blob/master/Dashboard.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
#CUSTOMER DETAILS
import json
import requests
import pandas as pd
import numpy as np
from pandas import json_normalize
from google.colab import auth
from google.cloud import bigquery
from google.cloud.bigquery.job import WriteDisposition
import getpass

import copy

maijaki_url_customers = "https://ppl-maijaki.sparkmeter.cloud/api/v0/customers"
maijaki_headers = {
     "Content-Type": "application/json",
     "Authentication-Token": ".eJwFwckRwCAIAMBeeIcZQS5ryeQhiP2XkN0XuiZHtuBIKxTngRFBWOZke10_5PDAihMVdTWm7SmerJqL-wxpJi34ftN3Eyo.Y9gTZw.nBM0bHvBvN_rDOZng3PKDEi42Hk"
}

mbiabet_url_customers = "https://ppl-mbiabet.sparkmeter.cloud/api/v0/customers"
mbiabet_headers = {
     "Content-Type": "application/json",
     "Authentication-Token": ".eJwFwcERgDAIBMBeeMsMkSD04vgALvRfgrsvaXfXEeGclbz1AZej2aKX-MDKlC7CsokHmjvr1lOQ0QlYeKqbbfp-GIsUhA.ZMFC2Q.69qla4gYX1I4hnvvHJxJc4tMpqg"
}

aninigi_url_customers = "https://ppl-aninigi.sparkmeter.cloud/api/v0/customers"
aninigi_headers = {
     "Content-Type": "application/json",
     "Authentication-Token": ".eJwFwckNwDAIBMBeeAfJB6yhligPg03_JWTmpYZpsOosIcGS53BIgjNmpI3yakUPrdnvUV8muZpB4QOeuy50d1XQ9wPrShOR.ZMFEHg.wOFXMGrYZzJ7T6OQh_-GPkWSYAQ"
}

lafiya_kpada_url_customers = "https://ppl-lafiya-kpada.sparkmeter.cloud/api/v0/customers"
lafiya_kpada_headers = {
     "Content-Type": "application/json",
     "Authentication-Token": ".eJwNx8kRwCAIAMBeeMcZRC5ryeShCP2XYPa3LwwUXMy7hYo3PkPbPF5_mXEFdZkKDyQVUaQhEqbZlmFhsyKoeqknfBfEchNJ.ZMFMBQ.sCBMXVMm2xb4HTBQMyXVAXr3MBE"
}

def get_obj(url, header):
    response = requests.get(url, headers=header)
    customer_obj = response.json()
    return customer_obj

aninigi_customers_default_obj = get_obj(aninigi_url_customers, aninigi_headers)
lafiya_kpada_customers_default_obj = get_obj(lafiya_kpada_url_customers, lafiya_kpada_headers)
maijaki_customers_default_obj = get_obj(maijaki_url_customers, maijaki_headers)
mbiabet_customers_default_obj = get_obj(mbiabet_url_customers, mbiabet_headers)

mbiabet_customers = copy.deepcopy(mbiabet_customers_default_obj)
aninigi_customers = copy.deepcopy(aninigi_customers_default_obj)
lafiya_kpada_customers = copy.deepcopy(lafiya_kpada_customers_default_obj)
maijaki_customers = copy.deepcopy(maijaki_customers_default_obj)

def func(obj):
  meters_obj = obj['meters'][0]
  for key,value in meters_obj.items():
    obj[key]=value
  del obj['meters']
  ground = obj['ground']
  obj['ground_id'] = ground['id']
  obj['ground_name'] = ground['name']
  del obj['ground']
  return obj

maijaki_customers_list = list(map(func, maijaki_customers["customers"]))
mbiabet_customers_list = list(map(func, mbiabet_customers["customers"]))
aninigi_customers_list = list(map(func, aninigi_customers["customers"]))
lafiya_kpada_customers_list = list(map(func, lafiya_kpada_customers["customers"]))

lafiya_kpada_df = pd.DataFrame.from_records(lafiya_kpada_customers_list)
aninigi_df = pd.DataFrame.from_records(aninigi_customers_list)
mbiabet_df = pd.DataFrame.from_records(mbiabet_customers_list)
maijaki_df = pd.DataFrame.from_records(maijaki_customers_list)

pd.set_option('display.max_columns', None)

maijaki_df['ground_name'] = 'Maijaki'
aninigi_df['ground_name'] = 'Aninigi'
mbiabet_df['ground_name'] = 'Mbiabet'
lafiya_kpada_df['ground_name'] = 'Lafiya-Kpada'

frames = [aninigi_df, lafiya_kpada_df, maijaki_df, mbiabet_df]
spark_customers = pd.concat(frames)
spark_customers = spark_customers[(spark_customers['active'] == True)]
spark_customers.rename(columns = {'credit_balance':'account_balance', 'id':'customer_id', 'current_tariff_name':'tariff_name',
                                  'serial':'meter_number', 'ground_id':'site_id', 'ground_name':'site_name'}, inplace = True)
spark_customers = spark_customers[ ['site_name','name', 'meter_number', 'tariff_name', 'phone_number', 'coords', 'customer_id',
                                    'account_balance','site_id','is_running_plan', 'last_energy','last_energy_datetime']]

spark_customers['meter_number'] = spark_customers['meter_number'].astype(str)
spark_customers['site_id'] = spark_customers['site_id'].astype(str)
spark_customers['account_balance'] = spark_customers['account_balance'].astype(str)
spark_customers['last_energy'] = spark_customers['last_energy'].astype(str)


# Constants
BASE_URL = "https://api.steama.co"
TOKEN_ENDPOINT = f"{BASE_URL}/get-token/"
CUSTOMERS_ENDPOINT = f"{BASE_URL}/customers/"

# Helper function to get API token
def get_api_token(username, password):
    payload = {"username": username, "password": password}
    response = requests.post(TOKEN_ENDPOINT, json=payload)
    if response.status_code == 200:
        return response.json()["token"]
    else:
        raise ValueError(f"Error obtaining API token: {response.status_code} - {response.text}")

# Helper function to get all customer data
def get_all_customer_data(api_token):
    headers = {"Authorization": f"Token {api_token}"}
    all_data = []
    url = CUSTOMERS_ENDPOINT
    while url:
        response = requests.get(url, headers=headers)
        if response.status_code == 200:
            data = response.json()
            all_data.extend(data["results"])
            url = data["next"]
        else:
            raise ValueError(f"Error: {response.status_code} - {response.text}")
    return all_data

# Helper function to process customer data
def process_customer_data(raw_data):
    df = pd.json_normalize(raw_data)
    df = df.drop(columns=["tags"])
    df = df.rename(columns={'utility_use_30_days.1': 'utility_use_30_days'})
    return df
# Helper function to load DataFrame into BigQuery

def main():
    # Get user credentials
    username = input("Please enter username: ")
    password = getpass.getpass("Please enter your password: ")

    # Obtain API token
    try:
        api_token = get_api_token(username, password)
        print(f"API token obtained: {api_token}")
    except ValueError as e:
        print(e)
        exit(1)

    # Fetch and process customer data
    try:
        raw_data = get_all_customer_data(api_token)
        customers_df = process_customer_data(raw_data)
        customers_df['name'] = customers_df['first_name'] + ' ' + customers_df['last_name']
        customers_df['coords'] = customers_df['latitude'] + ',' + customers_df['longitude']
        customers_df.rename(columns = {'id': 'customer_id', 'telephone': 'phone_number', 'service_based_tariff':'tariff_name',
                                       'site': 'site_id'}, inplace = True)

        customers_df['meter_number'] = " "
        customers_df['is_running_plan'] = ' '
        customers_df['last_energy'] = " "
        customers_df['last_energy'] = " "
        customers_df['last_energy_datetime'] = " "

        customers_df = customers_df[['site_name','name', 'meter_number', 'tariff_name', 'phone_number', 'customer_id',
                                    'account_balance','site_id']]
        customers_df['meter_number'] = customers_df['meter_number'].astype(str)
        customers_df['site_id'] = customers_df['site_id'].astype(str)

        return customers_df

    except ValueError as e:
        print(e)
        exit(1)

if __name__ == "__main__":
    customers_df = main()

customer_data_list = [spark_customers, customers_df]
customer_data = pd.concat(customer_data_list)
customer_data = customer_data.astype(str)
customer_data.info()

project_id = "minigrid-db-395810"
dataset_id = "Customers"

auth.authenticate_user()
client =bigquery.Client(project=project_id)

def func2(df):
  schema = []
  for col in df.columns:
    if df[col].dtype.name.lower() == "int64":
      col_type = "INTEGER"
    elif df[col].dtype.name.lower() == "float64":
      col_type = "FLOAT"
    elif df[col].dtype.name.lower() == "bool":
      col_type = "BOOLEAN"
    else:
      col_type = "STRING"
  schema.append(bigquery.SchemaField(col, col_type, mode="NULLABLE"))
  return schema

customers_table_id = "customers_data"
customers_table_ref = bigquery.TableReference.from_string(f"{project_id}.{dataset_id}.{customers_table_id}")
customers_table = bigquery.Table(customers_table_ref, schema= func2(customer_data))

job_config = bigquery.LoadJobConfig(write_disposition=bigquery.WriteDisposition.WRITE_TRUNCATE, create_disposition="CREATE_IF_NEEDED")
client.load_table_from_dataframe(customer_data, customers_table, job_config=job_config).result()

print("Data streamed to BigQuery successfully.")


Please enter username: d.momodu
Please enter your password: ··········
API token obtained: b464ef0bc14b14f9e070d9b5831087a55caaa7c2
<class 'pandas.core.frame.DataFrame'>
Int64Index: 1744 entries, 0 to 164
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   site_name             1744 non-null   object
 1   name                  1744 non-null   object
 2   meter_number          1744 non-null   object
 3   tariff_name           1744 non-null   object
 4   phone_number          1744 non-null   object
 5   coords                1744 non-null   object
 6   customer_id           1744 non-null   object
 7   account_balance       1744 non-null   object
 8   site_id               1744 non-null   object
 9   is_running_plan       1744 non-null   object
 10  last_energy           1744 non-null   object
 11  last_energy_datetime  1744 non-null   object
dtypes: object(12)
memory usage: 177.1+ KB
Data streamed to Bi

In [6]:
import requests
import pandas as pd
import numpy as np
from google.colab import auth
from google.cloud import bigquery
from google.cloud.bigquery.job import WriteDisposition
import getpass


aninigi_headers = {
     "Content-Type": "application/json",
     "Authentication-Token": ".eJwFwckNwDAIBMBeeAfJB6yhligPg03_JWTmpYZpsOosIcGS53BIgjNmpI3yakUPrdnvUV8muZpB4QOeuy50d1XQ9wPrShOR.ZMFEHg.wOFXMGrYZzJ7T6OQh_-GPkWSYAQ"
}

maijaki_headers = {
     "Content-Type": "application/json",
     "Authentication-Token": ".eJwFwckRwCAIAMBeeIcZQS5ryeQhiP2XkN0XuiZHtuBIKxTngRFBWOZke10_5PDAihMVdTWm7SmerJqL-wxpJi34ftN3Eyo.Y9gTZw.nBM0bHvBvN_rDOZng3PKDEi42Hk"
}

mbiabet_headers = {
     "Content-Type": "application/json",
     "Authentication-Token": ".eJwFwcERgDAIBMBeeMsMkSD04vgALvRfgrsvaXfXEeGclbz1AZej2aKX-MDKlC7CsokHmjvr1lOQ0QlYeKqbbfp-GIsUhA.ZMFC2Q.69qla4gYX1I4hnvvHJxJc4tMpqg"
}

lafiya_kpada_headers = {
     "Content-Type": "application/json",
     "Authentication-Token": ".eJwNx8kRwCAIAMBeeMcZRC5ryeShCP2XYPa3LwwUXMy7hYo3PkPbPF5_mXEFdZkKDyQVUaQhEqbZlmFhsyKoeqknfBfEchNJ.ZMFMBQ.sCBMXVMm2xb4HTBQMyXVAXr3MBE"
}

#Get table for sales accounts
maijaki_sales_accounts_url = "https://ppl-maijaki.sparkmeter.cloud/api/v0/sales-accounts"
aninigi_sales_accounts_url = "https://ppl-aninigi.sparkmeter.cloud/api/v0/sales-accounts"
mbiabet_sales_accounts_url = "https://ppl-mbiabet.sparkmeter.cloud/api/v0/sales-accounts"
lafiya_sales_accounts_url = "https://ppl-lafiya-kpada.sparkmeter.cloud/api/v0/sales-accounts"

project_id = "minigrid-db-395810"
dataset_id = "Agents"

def sales_account_df(sales_accounts_url, header):
    sales_response = requests.get(sales_accounts_url, headers=header)
    sales_response_obj = sales_response.json()

    sales_accounts_df = pd.DataFrame.from_records(sales_response_obj)
    sales_accounts_df = pd.json_normalize(sales_accounts_df.pop("accounts"))
    return sales_accounts_df

aninigi_agents_df = sales_account_df(aninigi_sales_accounts_url,aninigi_headers)
aninigi_agents_df['site_name'] = 'Aninigi'
maijaki_agents_df = sales_account_df(maijaki_sales_accounts_url,maijaki_headers)
maijaki_agents_df['site_name'] = 'Maijaki'
mbiabet_agents_df = sales_account_df(mbiabet_sales_accounts_url,mbiabet_headers)
mbiabet_agents_df['site_name'] = 'Mbiabet'
lafiya_agents_df = sales_account_df(lafiya_sales_accounts_url,lafiya_kpada_headers)
lafiya_agents_df['site_name'] = 'Lafiya-Kpada'
agents_frame = [aninigi_agents_df, maijaki_agents_df, mbiabet_agents_df, lafiya_agents_df]
agents_df = pd.concat(agents_frame)
agents_df.rename(columns = {'credit':'credit_balance'}, inplace = True)

agents_df = agents_df[['id', 'name', 'site_name', 'credit_balance']]
agents_df['credit_balance'] = agents_df['credit_balance'].astype(str)


#STEAMACO AGENTS
BASE_URL = "https://api.steama.co"
TOKEN_ENDPOINT = f"{BASE_URL}/get-token/"
AGENTS_ENDPOINT = f"{BASE_URL}/agents/"
PROJECT_ID = "minigrid-db"
TABLE_ID = "minigrid-db.raw_zone.steamaco_agents"

# Helper function to get API token
def get_api_token(username, password):
    payload = {"username": username, "password": password}
    response = requests.post(TOKEN_ENDPOINT, json=payload)
    if response.status_code == 200:
        return response.json()["token"]
    else:
        raise ValueError(f"Error obtaining API token: {response.status_code} - {response.text}")

# Helper function to get all customer data
def get_all_agents_data(api_token):
    headers = {"Authorization": f"Token {api_token}"}
    all_data = []
    url = AGENTS_ENDPOINT
    while url:
        response = requests.get(url, headers=headers)
        if response.status_code == 200:
            data = response.json()
            all_data.extend(data["results"])
            url = data["next"]
        else:
            raise ValueError(f"Error: {response.status_code} - {response.text}")
    return all_data

# Helper function to process customer data
def process_agents_data(raw_data):
    df = pd.json_normalize(raw_data)
    return df

def main():
    # Get user credentials
    username = input("Please enter username: ")
    password = getpass.getpass("Please enter your password: ")

    # Obtain API token
    try:
        api_token = get_api_token(username, password)
        print(f"API token obtained: {api_token}")
    except ValueError as e:
        print(e)
        exit(1)

    # Fetch and process customer data
    try:
        raw_data = get_all_agents_data(api_token)
        steamaco_agents = process_agents_data(raw_data)
        steamaco_agents['name'] = steamaco_agents['first_name'] + ' ' + steamaco_agents['last_name']
        return steamaco_agents
    except ValueError as e:
        print(e)
        exit(1)

if __name__ == "__main__":
    agents1_df = main()

agents1_df = agents1_df[['id', 'name', 'site_name', 'credit_balance']]

all_agents_frame = [agents_df, agents1_df]
agents = pd.concat(all_agents_frame)
agents = agents.astype(str)

dataset_id = "Agents"

def load_agents_dataframe_to_bigquery(df, project_id, dataset_id,table_id):
    client = bigquery.Client(project=project_id)
    schema = []
    for col in df.columns:
        if df[col].dtype.name.lower() == "int64":
            col_type = "INTEGER"
        elif df[col].dtype.name.lower() == "float64":
            col_type = "FLOAT"
        elif df[col].dtype.name.lower() == "bool":
            col_type = "BOOLEAN"
        else:
            col_type = "STRING"
        schema.append(bigquery.SchemaField(col, col_type, mode="NULLABLE"))

  # Convert DataFrame to BigQuery table
    table_ref = bigquery.TableReference.from_string(f"{project_id}.{dataset_id}.{table_id}")
    table = bigquery.Table(table_ref, schema= schema)

  # Stream the DataFrame data to the BigQuery tabl
    job_config = bigquery.LoadJobConfig(write_disposition=bigquery.WriteDisposition.WRITE_TRUNCATE, create_disposition="CREATE_IF_NEEDED")
    client.load_table_from_dataframe(agents, table, job_config=job_config).result()

    print("Data streamed to BigQuery successfully.")

def main():
    # Authenticate with Google Colab
    auth.authenticate_user()

    agents_table_id = 'agents_data'
    load_agents_dataframe_to_bigquery(agents, project_id, dataset_id, agents_table_id)
if __name__ == "__main__":
    main()

Please enter username: d.momodu
Please enter your password: ··········
API token obtained: b464ef0bc14b14f9e070d9b5831087a55caaa7c2
Data streamed to BigQuery successfully.


In [1]:
#TRANSACTION DATA

import requests
import pandas as pd
import numpy as np
from concurrent.futures import ThreadPoolExecutor, as_completed
import time
import getpass
from google.colab import auth
from google.cloud import bigquery
from google.cloud.bigquery.job import WriteDisposition


# Define your site IDs
aninigi_id = "bce9a7b0-57f1-4f34-a34e-159870aaecd5"
lafiya_kpada_id = "75503047-1a72-4582-bef3-1ce3f784a77d"
maijaki_id = "f59da0c8-c6fe-4041-a8f6-545572b5d7be"
mbiabet_id = "08f41ab4-afa0-4a2a-a5b9-3658afb93fdc"

# List of site IDs
site_ids = [aninigi_id, lafiya_kpada_id, maijaki_id, mbiabet_id]

# Headers with placeholders
headers = {
    'X-API-KEY': 'ng5JU0UhoQalLYg1tNnDXFRq_4T5-F92JPqvn9f819Y',
    'X-API-SECRET': 'mZbvDVjQ46x1Z*4L1KSYuOMce16ItHj^',
    'Content-Type': 'application/json'
}

def get_data(site_id):
    transaction_url = "https://www.sparkmeter.cloud/api/v0/organizations/29bba916-61b9-4f84-9a51-44478e184a80/data/historical"
    data = {
        "per_page": 100,
        "filters": {
            "sites": [site_id],
            "entity_types": ["transactions"]
        }
    }
    all_data = []
    while True:
        response = requests.post(transaction_url, json=data, headers=headers)
        if response.status_code == 200:
            response_data = response.json()
            all_data.extend(response_data["results"])

            if response_data.get('cursor'):
                data = {"cursor": response_data['cursor']}
                time.sleep(1.1)  # Added sleep here to respect rate limit
            else:
                break
        elif response.status_code == 429:  # Rate limit exceeded
            time.sleep(1.1)  # Added sleep here to respect rate limit
        else:
            print(f"Error: {response.status_code} - {response.text}")
            break

    return [(site_id, flatten_dict(record)) for record in all_data]

# Flatten the nested dictionary
def flatten_dict(d, parent_key='', sep='_'):
    items = []
    for k, v in d.items():
        new_key = f"{parent_key}{sep}{k}" if parent_key else k
        if isinstance(v, dict):
            items.extend(flatten_dict(v, new_key, sep=sep).items())
        else:
            items.append((new_key, v))
    return dict(items)

all_transactions = []

# Parallelizing the API calls
with ThreadPoolExecutor() as executor:
    future_to_site = {executor.submit(get_data, site_id): site_id for site_id in site_ids}
    for future in as_completed(future_to_site):
        site_id = future_to_site[future]
        try:
            data = future.result()
        except Exception as exc:
            print(f"{site_id} generated an exception: {exc}")
        else:
            all_transactions.extend(data)

# Convert to DataFrame
all_trans_df = pd.DataFrame.from_records([record for _, record in all_transactions])

all_trans_df['to_address_coords_lon'].fillna(0.0, inplace = True)
all_trans_df['to_address_coords_lat'].fillna(0.0, inplace = True)


all_trans_df.rename(columns = {'from_name':'agent_name', 'from_wallet_type':'wallet_type', 'from_id':'agent_id',
                               'state':'transaction_status','site':'site_id', 'to_serial':'meter_number',
                               'to_wallet_type':'wallet_type', 'to_tariff_name':'tariff_name', 'to_tariff_id':'tariff_id',
                               'to_type':'type', 'to_customer_name':'customer_name', 'to_customer_phone_number':'phone_number',
                               'to_customer_id':'customer_id', 'created':'transaction_date'}, inplace = True)

all_trans_df = all_trans_df[all_trans_df['transaction_status'] == 'processed']

all_trans_df.loc[all_trans_df['site_id'] == "bce9a7b0-57f1-4f34-a34e-159870aaecd5", 'site_name'] = 'Aninigi'
all_trans_df.loc[all_trans_df['site_id'] == "75503047-1a72-4582-bef3-1ce3f784a77d", 'site_name'] = 'Lafiya_kpada'
all_trans_df.loc[all_trans_df['site_id'] == "f59da0c8-c6fe-4041-a8f6-545572b5d7be", 'site_name'] = 'Maijaki'
all_trans_df.loc[~all_trans_df['site_id'].isin(["bce9a7b0-57f1-4f34-a34e-159870aaecd5", "75503047-1a72-4582-bef3-1ce3f784a77d", "f59da0c8-c6fe-4041-a8f6-545572b5d7be"]), 'site_name'] = 'Mbiabet'


pd.set_option('display.max_columns', None)
all_trans_df = all_trans_df[['site_name', 'origin', 'acct_type', 'source', 'amount','transaction_date', 'customer_name',
                              'phone_number','meter_number', 'tariff_name', 'agent_name', 'agent_id', 'id', 'tariff_id',
                              'site_id', 'customer_id', 'reference_id']]


#STEAMACO DATA
def get_api_token(username, password):
    token_endpoint = "https://api.steama.co/get-token/"
    payload = {"username": username, "password": password}
    response = requests.post(token_endpoint, json=payload)
    if response.status_code == 200:
        return response.json()["token"]
    else:
        raise ValueError(f"Error obtaining API token: {response.status_code} - {response.text}")

def get_transaction_data(api_token, base_url="https://api.steama.co", page_size=100):
    transactions_endpoint = f"{base_url}/transactions/"
    headers = {'Authorization': f"Token {api_token}"}
    all_data = []
    page = 1

    while True:
        url = f"{transactions_endpoint}?page={page}&page_size={page_size}"
        response = requests.get(url, headers=headers)

        if response.status_code == 200:
            data = response.json()
            all_data.extend(data["results"])

            if data["next"]:
                page += 1
            else:
                break
        else:
            print(f"Error: {response.status_code} - {response.text}")
            break

    transactions = pd.DataFrame.from_records(all_data)
    return transactions

def main():
    username = input("Please enter username: ")
    password = getpass.getpass("Please enter your password: ")

    try:
        api_token = get_api_token(username, password)
        print(f"API token obtained: {api_token}")
    except ValueError as e:
        print(e)
        exit(1)

    transactions = get_transaction_data(api_token)
    transactions = transactions.applymap(lambda x: str(x) if isinstance(x, list) else x)

    transactions['customer_name'] = transactions['customer_first_name'] + ' ' + transactions['customer_last_name']
    transactions['agent_name'] = transactions['agent_first_name'] + ' ' + transactions['agent_last_name']
    transactions = transactions[transactions['category'] == 'SUB']
    transactions.rename(columns = {'timestamp':'transaction_date','raw_message':'reference_id',
                                   'customer_telephone':'phone_number'}, inplace = True)
    transactions['origin'] = ' '
    transactions['acct_type'] = ' '
    transactions['source'] = ' '
    transactions['meter_number'] = ' '
    transactions['tariff_name'] = ' '
    transactions['tariff_id'] = ' '

    transactions = transactions[['site_name', 'origin', 'acct_type', 'source', 'amount','transaction_date', 'customer_name',
                                 'phone_number','meter_number', 'tariff_name','agent_name', 'agent_id', 'id','tariff_id',
                                 'site_id', 'customer_id', 'reference_id']]

    return transactions

if __name__ == "__main__":
    trans_df = main()

pd.set_option('display.max_columns', None)

trans_frame = [all_trans_df,trans_df]
transaction_data = pd.concat(trans_frame)
transaction_data = transaction_data.astype(str)

project_id = "minigrid-db-395810"
dataset_id = "Transactions"

auth.authenticate_user()
client =bigquery.Client(project=project_id)

def func2(df):
  schema = []
  for col in df.columns:
    if df[col].dtype.name.lower() == "int64":
      col_type = "INTEGER"
    elif df[col].dtype.name.lower() == "float64":
      col_type = "FLOAT"
    elif df[col].dtype.name.lower() == "bool":
      col_type = "BOOLEAN"
    else:
      col_type = "STRING"
  schema.append(bigquery.SchemaField(col, col_type, mode="NULLABLE"))
  return schema

transaction_table_id = "transaction_data"
transaction_table_ref = bigquery.TableReference.from_string(f"{project_id}.{dataset_id}.{transaction_table_id}")
transaction_table = bigquery.Table(transaction_table_ref, schema= func2(transaction_data))

job_config = bigquery.LoadJobConfig(write_disposition=bigquery.WriteDisposition.WRITE_TRUNCATE, create_disposition="CREATE_IF_NEEDED")
client.load_table_from_dataframe(transaction_data, transaction_table, job_config=job_config).result()

print("Data streamed to BigQuery successfully.")



<ipython-input-1-90bdcb3cd5c3>:99: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_trans_df.loc[all_trans_df['site_id'] == "bce9a7b0-57f1-4f34-a34e-159870aaecd5", 'site_name'] = 'Aninigi'


Please enter username: d.momodu
Please enter your password: ··········
API token obtained: b464ef0bc14b14f9e070d9b5831087a55caaa7c2
Data streamed to BigQuery successfully.
